In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
from astropy.table import Table
from colorplot import color_cut

In [ ]:
w_central = central_wavelength()

In [ ]:
split_converter = lambda s: np.array(s.split()).astype(float)
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))

cat = pd.read_csv('csv/minijpas.Flambda_photoz_gaia_7.csv', sep=',', header=1,
    converters={0: int, 1: int, 2: split_converter, 3: split_converter, 4: sum_flags,
    5: sum_flags})

cat = cat[np.array([len(x) for x in cat['FLUX_AUTO']]) != 0] # Drop bad rows due to bad query
cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
cat = cat.reset_index()

pm_flx = np.stack(cat['FLUX_AUTO'].to_numpy()).T * 1e-19
pm_err = np.stack(cat['FLUX_RELERR_AUTO'].to_numpy()).T * pm_flx

In [ ]:
# Function to drop sources
def make_masks(cat, nb_ind):
    # Mask sources with high photoz odds
    #mask_pz_odds = cat['odds'] < 0.98
    # Mask sources with proper motion according to Gaia (sigma > 3)
    parallax_sn = np.abs(cat['parallax'] / cat['parallax_error'])
    pmra_sn = np.abs(cat['pmra'] / cat['pmra_error'])
    pmdec_sn = np.abs(cat['pmdec'] / cat['pmdec_error'])
    mask_pmotion = (
        (np.sqrt(parallax_sn**2 + pmra_sn**2 + pmdec_sn**2) < 27.**0.5)
        | ( np.isnan(parallax_sn) | np.isnan(pmra_sn) | np.isnan(pmdec_sn) )
    )
    # Mask sources with SNR < 5 in the selected NB
    mask_snr = (pm_err / pm_flx)[nb_ind, :] < 0.2
    # Mask too bright gSDSS
    mask_b = pm_flx[-3, :] > mag_to_flux(15, 4750)
    
    mask_total = mask_pmotion & mask_snr & mask_b # & mask_pz_odds
    # print('Total masked: {}'.format(len(np.where(~mask_total)[0])))
    return mask_total.to_numpy()

In [ ]:
N_nb = 6 # Number of nb on each side of the central one
ew0min = 20

N_sources = len(cat['FLUX_AUTO'])
print(N_sources)

In [ ]:
nb_c_min = 6
nb_c_max = 50
line = np.zeros((nb_c_max - nb_c_min, N_sources)).astype(bool)
cont_est_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
cont_err_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
i = 0
for nb_c in range(nb_c_min, nb_c_max):
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(nb_c)
    cont_est, cont_err = stack_estimation(pm_flx, pm_err, nb_c, N_nb, False)
    line[i] = (
        (pm_flx[nb_c] - cont_est > 3 * (cont_err**2 + pm_err[nb_c]**2) ** 0.5)
        & (pm_flx[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )
    cont_est_Arr[i] = cont_est
    cont_err_Arr[i] = cont_err
    i += 1

In [ ]:
## QSO_find_lines
qso_LAE_Arr, qso_single_Arr, line_lya, line_other =\
    QSO_find_lines(pm_flx, pm_err, 6, 50, 30, 15, 6)

In [ ]:
print('QSO candidates found : {}'.format(len(qso_LAE_Arr)))
print('QSO candidates single: {}'.format(len(qso_single_Arr)))

In [ ]:
for j, src in enumerate(qso_LAE_Arr):
    fig = plt.figure(figsize=(10, 4))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

    for l in line_other[src]:
        ax.errorbar(w_central[l],
            cont_est_Arr[l - 6, src],
            yerr=cont_err_Arr[l - 6, src],
            fmt='s', c='black', markersize=8, capsize=3, capthick=2, elinewidth=2)

    ax.errorbar(w_central[line_lya[src]],
        cont_est_Arr[line_lya[src] - 6, src],
        yerr=cont_err_Arr[line_lya[src] - 6, src],
        fmt='s', c='black', markersize=8, capsize=4, capthick=2, elinewidth=2,
        label='Continuum estimate')
    ax.legend(fontsize=15)
    plt.show()

    z_src = w_central[line_lya[src]] / 1215.67 - 1
    z_src_e = nb_fwhm(line_lya[src]) / 1215.67 / 2

    print('z_nb = {0:0.2f} +- {1:0.2f}'.format(z_src, z_src_e))
    print('z_sp = {0:0.2f} ({1})'.format(cat.zsp[src], cat.spCl[src]))
    print('z_pz = {0:0.2f}, odds = {1:0.2f}'.format(cat.PHOTOZ[src], cat.ODDS[src]))
    print('RA, DEC = {}, {}'.format(cat.ALPHA_J2000[src], cat.DELTA_J2000[src]))

    if j == 15: break

In [ ]:
spec = Table.read('../SDSS_Spectra/spec-7030-56448-0408.fits', format='fits')
spec_f = spec['flux'] * 1e-17
spec_m = spec['model'] * 1e-17
spec_w = 10 ** spec['loglam']

In [ ]:
src = np.where(cat.NUMBER + cat.TILE_ID*100000 == 247001135)[0]

fig= plt.figure(figsize=(15, 5))

#ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
ax = plt.gca()
ax.plot(spec_w, spec_f, c='black', alpha=0.3, zorder=-99)
ax.plot(spec_w, spec_m, c='black', alpha=0.8, zorder=-99)
ax.set_xlim((3200, 9500))
ax.set_ylim((-3e-17, 4e-17))
ax.set_ylabel(r'$f_\lambda$ ( erg cm $^{-2}$ s$^{-1}$ $\AA^{-1}$)', fontsize=15)
ax.set_xlabel(r'$\lambda$ ($\AA$)', fontsize=15)
ax.legend(['SDSS spectrum', 'Model fit', 'miniJPAS data'], fontsize=15, loc=1)

plt.show()

In [ ]:
mag = flux_to_mag(pm_flx, w_central[-2])

In [ ]:
fig, axs = plt.subplots(5, figsize=(8, 15), sharex=True)

for i, nb_c in enumerate([8, 11, 14, 17, 26]):
    cont_est, cont_err = stack_estimation(pm_flx, pm_err, nb_c, 6, True)

    mag_est = flux_to_mag(cont_est, w_central[nb_c])

    bbnb = pm_flx[nb_c] - cont_est

    this_z = w_central[nb_c] / 1215.67 - 1

    line = np.where(
        (bbnb > 3 * (cont_err**2 + pm_err[nb_c]**2) ** 0.5)
        & (bbnb > (1 + this_z) * 30 * cont_est / 145)
    )


    bbnbmag = mag_est - mag[nb_c]
    bbnbmag = bbnbmag[~np.isnan(bbnbmag)]

    ccut = color_cut(30, nb_c) + np.median(bbnbmag)

    ax = axs[i]

    ax.scatter(mag[nb_c], mag_est - mag[nb_c], marker='.')
    ax.scatter(mag[nb_c, line], mag_est[line] - mag[nb_c, line])
    ax.plot(np.linspace(16, 27, 200), np.ones(200)*ccut, c='red', label='EW$_0$ = 30 $\AA$')

    ax.set_ylim((-2, 4))
    ax.set_xlim((17, 26))

    ax.set_xlabel(load_filter_tags()[nb_c])
    ax.set_ylabel('cont_estimation - ' + load_filter_tags()[nb_c])

plt.show()